In [1]:
# Import libraries
import pandas as pd
import numpy as np
import csv

In [2]:
# Import data from file, split data by tab and names the columns
#data = pd.read_csv('train.tsv', delimiter="\t", header = None, names = ['word', 'mutation'])
train = pd.read_csv(r"train.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE, names = ['word', 'mutation'])
test = pd.read_csv(r"test.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE, names = ['word', 'mutation'])

In [3]:
# Check data columns and type
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   word      object
 1   mutation  object
dtypes: object(2)
memory usage: 152.6+ MB


In [4]:
# Look at the top 10 rows of data
train.head(10)

,word,mutation
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N
5,gaeilge,N
6,ag,N
7,na,N
8,gardaí,N
9,ná,N


In [5]:
# Inspecting Null values on train data
train.isnull().sum()

word        27
mutation     0
dtype: int64

In [6]:
# Inspecting Null values on test data
test.isnull().sum()

word        5
mutation    0
dtype: int64

In [7]:
# Removing null records
train.dropna(inplace=True)
test.dropna(inplace=True)

In [8]:
# Check data rows and columns
print(train.shape)
print(test.shape)

(9999973, 2)
(999995, 2)


In [9]:
# Find unique words
train.word.nunique(), train.mutation.nunique()

(171513, 5)

In [10]:
train.word.value_counts()

a              402501
<S>            395922
an             386495
.              367965
,              328017
                ...  
céatatan            1
eoireachta          1
d'iompóradh         1
máidin              1
fíormheas           1
Name: word, Length: 171513, dtype: int64

In [13]:
len(train)

9999973

In [14]:
# Split train data into training and validation set (80:20)
data_train = train[((len(train) // 10) * 2):]
data_val = train[:(((len(train)+1) // 10) * 2)]

In [15]:
print(data_train.shape)
print(data_val.shape)

(7999979, 2)
(1999994, 2)


In [17]:
data_train.head()

,word,mutation
1999999,tú,S
2000000,a,N
2000001,dícháiliú,S
2000002,ó,N
2000003,capall,S


In [18]:
data_val.head()

,word,mutation
0,ansin,N
1,),N
2,tá,N
3,níos,N
4,lú,N


In [19]:
# Turn data to list
data_trainlist = list(data_train.itertuples(index=False, name=None))
data_vallist = list(data_val.itertuples(index=False, name=None))
data_testlist = list(test.itertuples(index=False, name=None))

In [20]:
# Show mutation in training data
mutation = {mut for word,mut in data_trainlist}
print(len(mutation))
print(mutation)
vocab = {word for word,mut in data_trainlist}

5
{'U', 'N', 'T', 'H', 'S'}


In [21]:
# Emission Probability
def word_given_mut(word, mut, train_bag = data_trainlist):
    count_mut = len(train_bag)
    w_given_mut_list = [pair[0] for pair in train_bag if pair[0]==word]
    count_w_given_mut = len(w_given_mut_list)
    
    return (count_w_given_mut, count_mut)

In [23]:
# Transition Probability
def t2_given_t1(t2, t1, train_bag = data_trainlist):
    mutation = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in mutation if t==t1])
    count_t2_t1 = 0
    for index in range(len(mutation)-1):
        if mutation[index]==t1 and mutation[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [24]:
# Transition matrix of mutation
 
mut_matrix = np.zeros((len(mutation), len(mutation)), dtype='float32')
for i, t1 in enumerate(list(mutation)):
    for j, t2 in enumerate(list(mutation)): 
        mut_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(mut_matrix)

[[7.01960525e-04 9.60426986e-01 6.10400457e-05 7.01960525e-04
  3.81080620e-02]
 [3.79965566e-02 8.43446314e-01 4.05881787e-03 9.11115296e-03
  1.05386995e-01]
 [1.07323009e-04 9.38611269e-01 1.07323009e-04 2.71884957e-03
  5.84552623e-02]
 [8.09590507e-04 9.57558751e-01 1.55690490e-04 2.42877170e-03
  3.90471742e-02]
 [1.21055415e-03 9.45368290e-01 6.42544692e-05 1.59736606e-03
  5.17595448e-02]]


In [25]:
# Matrix to dataframe
mut_df = pd.DataFrame(mut_matrix, columns = list(mutation), index=list(mutation))
display(mut_df)

,U,N,T,H,S
U,0.000702,0.960427,0.000061,0.000702,0.038108
N,0.037997,0.843446,0.004059,0.009111,0.105387
T,0.000107,0.938611,0.000107,0.002719,0.058455
H,0.000810,0.957559,0.000156,0.002429,0.039047
S,0.001211,0.945368,0.000064,0.001597,0.051760


In [26]:
# Define Viterbi algorithm 
def Viterbi(words, train_bag = data_trainlist):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for mut in T:
            if key == 0:
                transition_p = mut_df.loc['N', mut]
            else:
                transition_p = mut_df.loc[state[-1], mut]
                 
            emission_p = word_given_mut(words[key], mut)[0]/word_given_mut(words[key], mut)[1] if (word_given_mut(words[key], mut)[1]) != 0 else 0
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # Maximum probability
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

**Test data**

In [ ]:
# Testing the algorithm
mut_seq = Viterbi(data_testlist)
 
# Find accuracy
check = [i for i, j in zip(mut_seq, data_testlist[1]) if i == j] 
 
accuracy = len(check)/len(mut_seq)
print('Accuracy : ',accuracy)

Reference:

https://ireneli.eu/2016/04/30/nlp-02-a-trigram-hidden-markov-model-python/

https://www.mygreatlearning.com/blog/pos-tagging/#sh5